In [9]:
# ---------------- Imports ----------------
import os
import json
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch
import yaml
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm



In [ ]:
# ---------------- Config ----------------
with open("../../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

data_path = config["paths"]["proj_store"]
models_folderpath = config["paths"]["models"]

generated_blocks_file = "2025_09_01_15_46_09_llama_3_8b_instruct_generated_blocks"

model_choice = f"{models_folderpath}/sentence-transformers/all-MiniLM-L12-v2"
domain_index_file = f"{data_path}/indexes/domain_index_by_id.json"

# --- configuration---
merged_jsonl_path = f"{data_path}/output/generated_blocks/{generated_blocks_file}.jsonl"
results_root = f"{data_path}/results/interaction_metrics/generated_dataset"


# ------------------------
# MODEL INIT
# ------------------------
# Load sentence embedding model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
embedding_model = SentenceTransformer(model_choice, device=device)






cuda


In [3]:
# ------------------------
# Load JSONL
# ------------------------
def load_merged_jsonl(path: str):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            s = line.strip()
            if not s:
                continue
            obj = json.loads(s)
            assert "block_id" in obj and "context_turns" in obj and "response_turns" in obj, f"{path}:{i} missing keys"
            rows.append(obj)
    return rows


def sequence_from_record(rec: dict, resp_label: str):
    # take turns 1..4 from context_turns
    ctx = sorted(rec["context_turns"], key=lambda x: x.get("turn", 0))[:4]
    # get response (turn 5)
    resp = next((r for r in rec["response_turns"] if r.get("label") == resp_label), None)
    if resp is None:
        return None

    turns = []
    for t in ctx:
        turns.append({
            "turn": t.get("turn"),
            "role": t.get("role", "").lower(),
            "label": t.get("label", "real"),
            "utterance": t.get("utterance", "").strip()
        })
    turns.append({
        "turn": 5,
        "role": resp.get("role", "").lower(),
        "label": resp_label,
        "utterance": resp.get("utterance", "").strip()
    })

    turns = sorted(turns, key=lambda x: x["turn"])
    if [t["turn"] for t in turns] != [1, 2, 3, 4, 5]:
        return None

    return {
        "block_id": rec["block_id"],
        "turns": turns
    }


# ------------------------
# Load Domain Mapping
# ------------------------
with open(domain_index_file, "r", encoding="utf-8") as f:
    _raw_domain_ix = json.load(f)

dialogueid_to_domain = {row["dialogue_id"]: row["domain"] for row in _raw_domain_ix}

def get_domain_from_block(block_id: str) -> str:
    # block_id looks like "jsc-oral-history-00128:23"
    dialogue_id = block_id.split(":")[0]
    return dialogueid_to_domain.get(dialogue_id, "Unknown")



In [12]:
from scipy.stats import entropy

def kl_divergence(p, q):
    """Safe KL divergence with small epsilon to avoid log(0)."""
    eps = 1e-10
    p = np.asarray(p) + eps
    q = np.asarray(q) + eps
    p = p / p.sum()
    q = q / q.sum()
    return entropy(p, q)  # KL(P || Q)

def js_divergence(p, q):
    """Symmetric Jensen–Shannon divergence."""
    m = 0.5 * (p + q)
    return 0.5 * (kl_divergence(p, m) + kl_divergence(q, m))



def compute_distribution_kl(df, metrics, labels, out_dir, timestamp, domain):
    results = []
    # build histograms on common bins per metric
    for metric in metrics:
        all_vals = df[metric].dropna().values
        if len(all_vals) == 0:
            continue
        bins = np.linspace(all_vals.min(), all_vals.max(), 30)

        # histograms per label
        hists = {}
        for lab in labels:
            vals = df[df["Label"] == lab][metric].dropna().values
            if len(vals) == 0:
                continue
            hist, _ = np.histogram(vals, bins=bins, density=True)
            hists[lab] = hist

        # pairwise KL
        lab_pairs = [("real", "fine_tuned"), ("real", "prompted"), ("fine_tuned", "prompted")]
        for a, b in lab_pairs:
            if a in hists and b in hists:
                kl_ab = kl_divergence(hists[a], hists[b])
                kl_ba = kl_divergence(hists[b], hists[a])
                results.append({
                    "Domain": domain,
                    "Metric": metric,
                    "KL_{}_to_{}".format(a, b): kl_ab,
                    "KL_{}_to_{}".format(b, a): kl_ba
                })

    if results:
        out_df = pd.DataFrame(results)
        out_csv = f"{out_dir}/{timestamp}_kl_divergence_{domain}.csv"
        out_df.to_csv(out_csv, index=False)
        print(f"[info] wrote KL divergence results for domain={domain} to {out_csv}")


In [4]:

# ------------------------
# Main
# ------------------------

timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
out_dir = f"{results_root}/response_context_similarity"
Path(out_dir).mkdir(parents=True, exist_ok=True)

# Load data
rows = load_merged_jsonl(merged_jsonl_path)
seqs = []
labels = ["real", "fine_tuned", "prompted"]
for rec in rows:
    for lab in labels:
        s = sequence_from_record(rec, lab)
        if s is not None:
            s["domain"] = get_domain_from_block(s["block_id"])
            seqs.append(s)
print(f"[info] built {len(seqs)} sequences from {len(rows)} blocks")

# Compute similarities
similarity_records = []
for dialogue in tqdm(seqs, desc="Response-Context Similarity"):
    turns = dialogue["turns"]
    context_utts = [t["utterance"] for t in turns[:4]]
    resp_utt = turns[4]["utterance"]

    if not resp_utt.strip():
        continue

    embeddings = embedding_model.encode(context_utts + [resp_utt],
                                        batch_size=32, show_progress_bar=False, device=device)
    context_embs = embeddings[:4]
    resp_emb = embeddings[4]

    sims = [cosine_similarity([resp_emb], [ctx])[0][0] for ctx in context_embs]

    similarity_records.append({
        "BlockID": dialogue["block_id"],
        "Domain": dialogue["domain"],
        "Label": dialogue["turns"][-1]["label"],
        "Sim_Turn_4": sims[3],
        "Sim_Turn_3": sims[2],
        "Sim_Turn_2": sims[1],
        "Sim_Turn_1": sims[0],
        "Sim_All_Avg": float(np.mean(sims))
    })

# Save raw dataframe
similarity_df = pd.DataFrame(similarity_records)
csv_out = f"{out_dir}/{timestamp}_response_context_similarity.csv"
similarity_df.to_csv(csv_out, index=False)
print(f"[info] wrote raw similarities to {csv_out}")


[info] built 28071 sequences from 9357 blocks


Response-Context Similarity:   0%|          | 1/28071 [00:00<1:00:05,  7.79it/s]

Response-Context Similarity: 100%|██████████| 28071/28071 [04:17<00:00, 109.15it/s]


[info] wrote raw similarities to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity.csv


In [5]:

# Generate per-domain summary tables
domains = similarity_df["Domain"].unique()
for dom in domains:
    subset = similarity_df[similarity_df["Domain"] == dom]
    if subset.empty:
        continue

    summary_rows = []
    for label in labels:
        lab_subset = subset[subset["Label"] == label]
        if lab_subset.empty:
            continue
        summary_rows.append({
            "Label": label,
            "avg_sim_turn_4": lab_subset["Sim_Turn_4"].mean(),
            "avg_sim_turn_3": lab_subset["Sim_Turn_3"].mean(),
            "avg_sim_turn_2": lab_subset["Sim_Turn_2"].mean(),
            "avg_sim_turn_1": lab_subset["Sim_Turn_1"].mean(),
            "avg_sim_turn_all": lab_subset["Sim_All_Avg"].mean()
        })

    summary_df = pd.DataFrame(summary_rows)
    summary_csv_out = f"{out_dir}/{timestamp}_response_context_similarity_summary_{dom}.csv"
    summary_df.to_csv(summary_csv_out, index=False)
    print(f"[info] wrote summary averages for domain={dom} to {summary_csv_out}")



[info] wrote summary averages for domain=oral_history to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_oral_history.csv
[info] wrote summary averages for domain=academic_interviews to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_academic_interviews.csv
[info] wrote summary averages for domain=judicial_dialogue to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_judicial_dialogue.csv
[info] wrote summary averages for domain=journalistic_interviews to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_journalistic_interviews.csv


In [10]:


# Per-domain summaries and plots
domains = similarity_df["Domain"].unique()
for dom in domains:
    subset = similarity_df[similarity_df["Domain"] == dom]
    if subset.empty:
        continue

    summary_rows = []
    for label in labels:
        lab_subset = subset[subset["Label"] == label]
        if lab_subset.empty:
            continue
        summary_rows.append({
            "Label": label,
            "avg_sim_turn_4": lab_subset["Sim_Turn_4"].mean(),
            "avg_sim_turn_3": lab_subset["Sim_Turn_3"].mean(),
            "avg_sim_turn_2": lab_subset["Sim_Turn_2"].mean(),
            "avg_sim_turn_1": lab_subset["Sim_Turn_1"].mean(),
            "avg_sim_turn_all": lab_subset["Sim_All_Avg"].mean()
        })

        # Plot distributions for this domain × label
        fig, axes = plt.subplots(1, 5, figsize=(20, 4))
        metrics = ["Sim_Turn_4", "Sim_Turn_3", "Sim_Turn_2", "Sim_Turn_1", "Sim_All_Avg"]
        for ax, metric in zip(axes, metrics):
            ax.hist(lab_subset[metric], bins=20, alpha=0.7)
            ax.set_title(f"{metric} ({label})")
            ax.set_xlabel("cosine similarity")
            ax.set_ylabel("count")
        plt.tight_layout()
        plot_out = f"{out_dir}/{timestamp}_distribution_{dom}_{label}.png"
        plt.savefig(plot_out)
        plt.close()
        print(f"[info] wrote distribution plot {plot_out}")

    summary_df = pd.DataFrame(summary_rows)
    summary_csv_out = f"{out_dir}/{timestamp}_response_context_similarity_summary_{dom}.csv"
    summary_df.to_csv(summary_csv_out, index=False)
    print(f"[info] wrote summary averages for domain={dom} to {summary_csv_out}")




[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_real.png
[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_fine_tuned.png
[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_prompted.png
[info] wrote summary averages for domain=oral_history to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_oral_history.csv
[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_academic_interviews_real.png
[info] wrote distributio

In [13]:
from scipy.stats import entropy

def kl_divergence(p, q):
    """Safe KL divergence with small epsilon to avoid log(0)."""
    eps = 1e-10
    p = np.asarray(p) + eps
    q = np.asarray(q) + eps
    p = p / p.sum()
    q = q / q.sum()
    return entropy(p, q)  # KL(P || Q)

def js_divergence(p, q):
    """Symmetric Jensen–Shannon divergence."""
    m = 0.5 * (p + q)
    return 0.5 * (kl_divergence(p, m) + kl_divergence(q, m))


# Per-domain summaries, plots, and divergences
domains = similarity_df["Domain"].unique()
for dom in domains:
    subset = similarity_df[similarity_df["Domain"] == dom]
    if subset.empty:
        continue

    summary_rows = []
    for label in labels:
        lab_subset = subset[subset["Label"] == label]
        if lab_subset.empty:
            continue
        summary_rows.append({
            "Label": label,
            "avg_sim_turn_4": lab_subset["Sim_Turn_4"].mean(),
            "avg_sim_turn_3": lab_subset["Sim_Turn_3"].mean(),
            "avg_sim_turn_2": lab_subset["Sim_Turn_2"].mean(),
            "avg_sim_turn_1": lab_subset["Sim_Turn_1"].mean(),
            "avg_sim_turn_all": lab_subset["Sim_All_Avg"].mean()
        })

        # Plot distributions for this domain × label
        fig, axes = plt.subplots(1, 5, figsize=(20, 4))
        metrics = ["Sim_Turn_4", "Sim_Turn_3", "Sim_Turn_2", "Sim_Turn_1", "Sim_All_Avg"]
        for ax, metric in zip(axes, metrics):
            ax.hist(lab_subset[metric], bins=20, alpha=0.7)
            ax.set_title(f"{metric} ({label})")
            ax.set_xlabel("cosine similarity")
            ax.set_ylabel("count")
        plt.tight_layout()
        plot_out = f"{out_dir}/{timestamp}_distribution_{dom}_{label}.png"
        plt.savefig(plot_out)
        plt.close()
        print(f"[info] wrote distribution plot {plot_out}")

    # Save summary averages
    summary_df = pd.DataFrame(summary_rows)
    summary_csv_out = f"{out_dir}/{timestamp}_response_context_similarity_summary_{dom}.csv"
    summary_df.to_csv(summary_csv_out, index=False)
    print(f"[info] wrote summary averages for domain={dom} to {summary_csv_out}")

    # ---- Divergence analysis ----
    metrics = ["Sim_Turn_4", "Sim_Turn_3", "Sim_Turn_2", "Sim_Turn_1", "Sim_All_Avg"]
    label_pairs = [("real", "fine_tuned"), ("real", "prompted"), ("fine_tuned", "prompted")]
    div_results = []

    for metric in metrics:
        # Build bins for this metric (common to all labels in domain)
        all_vals = subset[metric].dropna().values
        if len(all_vals) == 0:
            continue
        bins = np.linspace(all_vals.min(), all_vals.max(), 30)

        # Histograms per label
        hists = {}
        for lab in labels:
            vals = subset[subset["Label"] == lab][metric].dropna().values
            if len(vals) == 0:
                continue
            hist, _ = np.histogram(vals, bins=bins, density=True)
            hists[lab] = hist

        # Pairwise divergences
        for a, b in label_pairs:
            if a in hists and b in hists:
                p, q = hists[a], hists[b]
                div_results.append({
                    "Domain": dom,
                    "Metric": metric,
                    "Pair": f"{a}_vs_{b}",
                    "KL_a_to_b": kl_divergence(p, q),
                    "KL_b_to_a": kl_divergence(q, p),
                    "JSD": js_divergence(p, q)
                })

    if div_results:
        div_df = pd.DataFrame(div_results)
        div_csv_out = f"{out_dir}/{timestamp}_distribution_divergence_{dom}.csv"
        div_df.to_csv(div_csv_out, index=False)
        print(f"[info] wrote divergence analysis for domain={dom} to {div_csv_out}")


[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_real.png
[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_fine_tuned.png
[info] wrote distribution plot /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_oral_history_prompted.png
[info] wrote summary averages for domain=oral_history to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_response_context_similarity_summary_oral_history.csv
[info] wrote divergence analysis for domain=oral_history to /data/sequential_ieas//results/interaction_metrics/generated_dataset/response_context_similarity/2025_09_22_22_09_05_distribution_divergence_oral_history.